In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import math
from datetime import datetime, timedelta
from typing import Union

import numpy as np
import pandas as pd

import formulas
import dataframes
import analysis

import matplotlib.pyplot as plt

In [ ]:
TAX_APPRAISAL_VAL = 500*10**3
DOWN_PAYMENT = TAX_APPRAISAL_VAL * .20
# DOWN_PAYMENT = 70*10**3
PRINCIPAL = TAX_APPRAISAL_VAL - DOWN_PAYMENT
LOAN_APR = .033
PAYMENTS_N = 12
YEARS = 30

PMI_RATE = 0.005
TAX_APR = 0.0182
TAX_APPRAISAL_VAL_GROWTH_RATE = 0.052

In [ ]:
%%timeit -n 100 -r 2
df = dataframes.return_on_investment(
    TAX_APPRAISAL_VAL,
    DOWN_PAYMENT,
    LOAN_APR,
    num_years=YEARS,
    pmi_rate=PMI_RATE,
    property_tax_rate=TAX_APR,
    apprasial_growth=TAX_APPRAISAL_VAL_GROWTH_RATE
)

df.round(2)

In [ ]:
analysis.annualized_return_vs_growth(
    initial_value=TAX_APPRAISAL_VAL,
    loan_interest_rate=LOAN_APR,
    growth_rates=np.arange(0.01, 0.15, 0.005),
    num_years=YEARS,
    pmi_rate=PMI_RATE,
    property_tax_rate=TAX_APR,
    figsize=(19.2, 10.8),
    facecolor='white'
)

In [ ]:
# %%timeit -n 2 -r 1
df = analysis.monthly_payment_vs_down_pmt(
    loan_interest_rate=LOAN_APR,
    num_years=YEARS,
    pmi_rate=PMI_RATE,
    property_tax_rate=TAX_APR,
    figsize=(19.2, 10.8),
    facecolor='white'
)
df

In [ ]:
df = pd.DataFrame(columns=pd.Index(np.arange(200, 650, 50)),
                  index=pd.Index(np.arange(25, 205, 5), name='down_pmt'))
df.stack()

# Property Tax

<span style="color:red">
</span>

In [ ]:
tax_df = dataframes.property_tax_amortization(TAX_APPRAISAL_VAL, TAX_APR, YEARS + 1, TAX_APPRAISAL_VAL_GROWTH_RATE)
display(tax_df.round(2))

# Mortgage Amortization

In [ ]:
df = dataframes.mortgage_amortization(PRINCIPAL, DOWN_PAYMENT, LOAN_APR, YEARS)
display(df.round(2))

In [ ]:
[v for v in np.arange(200, 550, 50) * 10 ** 3]

In [ ]:
df = pd.DataFrame(
    columns=pd.Index(np.arange(200, 550, 50) * 10 ** 3),
    index=pd.Index(np.arange(0.05, 0.35, 0.01) * TAX_APPRAISAL_VAL, name='down_pmt')
)

vroi = np.vectorize(
    lambda val, dp: analysis.return_on_investment(
        initial_value=val,
        down_payment=dp,
        loan_interest_rate=LOAN_APR,
        num_years=YEARS,
        pmi_rate=PMI_RATE,
        property_tax_rate=TAX_APR,
        apprasial_growth=TAX_APPRAISAL_VAL_GROWTH_RATE)['total monthly payment'].iloc[1] / 10 ** 3
)
vroi([df.columns.values, df.index.values])
df

In [ ]:
# %%timeit -n 10 -r 1
df = pd.DataFrame(
    columns=pd.Index(np.arange(200, 650, 50)),
    index=pd.Index(np.arange(0.05, 0.35, 0.01) * TAX_APPRAISAL_VAL / 10**3, name='down_pmt')
)

for idx in np.arange(df.shape[0]):
    for col in np.arange(df.shape[1]):
        df.iloc[idx, col] = analysis.return_on_investment(
            initial_value=df.columns[col] * 10 ** 3,
            down_payment=df.index[idx] * 10**3,
            loan_interest_rate=LOAN_APR,
            num_years=YEARS,
            pmi_rate=PMI_RATE,
            property_tax_rate=TAX_APR,
            apprasial_growth=TAX_APPRAISAL_VAL_GROWTH_RATE)['total monthly payment'].iloc[1] / 10**3
df

In [ ]:
fig, ax = plt.subplots(figsize=(19.2, 10.8))
ax.plot(df, label=[f'${v:.0f}k' for v in df.columns.values])
ax.grid(True)
ax.set_ylabel('Initial Monthly Payment')
ax.xaxis.set_major_formatter('${x:.0f}k')
ax.set_xlabel('Down Payment')
ax.yaxis.set_major_formatter('${x:.1f}k')
ax.legend(title='Home Value')

In [ ]:
df